# Notebook for Generating Graphics for the Paper

We used this file to generate the graphics for the paper. Because placement of legends, etc.
needs to vary so much, depending on how the data comes out, there can't be a
universal procedure. So we simply edit this file as needed to produce the plots.

Checked and cleaned up a bit by SOK on 2017-09-06.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pwd

In [ ]:
print(os.listdir())

In [ ]:
theSpot = '../outputs/' + 'Run2017-01-10singlecaste2017-01-11 11:18:58.752406'
#'Run2017-01-10singlecaste2017-01-11 10:55:34.006462'
#'Run2017-01-10singlecaste2017-01-10 12:24:23.223554'
#'Run2016-09-12'

print(os.getcwd())
os.chdir(theSpot)
print(os.getcwd())
print(os.listdir())

### Initialize

Assumes that we have above set the path, the cwd, to the directory that
holds RunDataMulti and Graphics. We'll read from RunDataMulti and write 
to Graphics.

In [ ]:
pathToRunData = 'RunDataMulti'
outputFolder =  'Graphics'

if not os.path.isdir(outputFolder):
    os.mkdir(outputFolder)

# These are the parameter values we swept in generating the
# output files.
aggressionRes= [0.5] #[0.4,0.3,0.1] #[1.0, 0.95, 0.9, 0.85]
metCostFactors=[0.5] #[0.4,0.3,0.1] #[1.0, 0.95, 0.9, 0.85]
resourceHonestAppropriations=[0.5] #[0.4,0.3,0.1] #[1.0, 0.95, 0.9, 0.85]
resourceDisHonestAppropriations=[0.5] #[0.4,0.3,0.1] #[1.0, 0.95, 0.9, 0.85]

#### end of data to be entgered by hand.
id0name = "Aggression Resources"
id1name= "Metabolic Cost Factors"
id2name = "Appropriations from Honest Wasps"
id3name =  "Appropriations from Dishonest Wasps"

numberOfRuns = len(aggressionRes)*len(metCostFactors)*\
    len(resourceHonestAppropriations)*len(resourceDisHonestAppropriations)
print(numberOfRuns)

In [ ]:
def makeExperimentParameterDict():
    '''
    Allows us to recover the parameter values used for the
    the experiment number, which is displayed on the
    input file, the .csv file in RunDataMulti.
    
    Creates a dictionary in which the keys are the
    experiment numbers, starting with 0, and the
    values are the parameter settings used, in a 
    4-tuple. i0 = aggressionRes. Initial resources allotted
    per castes. These numbers reduce initial values by
    percentages.
    i1 = metCostFactors. Metabolic cost factors, how 
    expensive it is to run away. These numbers start at
    1 and decline. The three castes have different initial
    resources. Metabolic costs are figured as percentages.
    These numbers reduce the initial levels by a percentage.
    So, it becomes cheaper to run away.
    i2 = resourceHonestAppropriations. How much you
    collect from honest wasps when they lose. Again, these numbers
    reudce by a percentage the program initial values.
    i3 = resourceDisHonestAppropriations. i2 but from liars.
    *********** ==>>
    So, the output is like this: A diction with keys: exeriment numbers 0-255,
    and values: 4-tuples of parameter values, as described above.
    <<==*************
    '''
    toReturn = {}
    experimentnumber = 0
    for i0 in aggressionRes:
        for i1 in metCostFactors:
            for i2 in resourceHonestAppropriations:
                for i3 in resourceDisHonestAppropriations:
                    toReturn[experimentnumber] = (i0,i1,i2,i3)
                    experimentnumber += 1
    return toReturn
experimetParameterDict = makeExperimentParameterDict()
# ok, works
# print(experimetParameterDict)

In [ ]:
def processFileList(fileList,plotVariable,parameterValues,experimentNumber):
    '''
    We extract the say 30 replication files from the repositoryin main() 
    below. Here we get the list and we make a DataFrame of each file.
    Then we process each DataFrame to get a graph.
    
    2016-08-27 sok: cleaned up this procedure to eliminate the
    problem of having many open graphics files.
    '''
    dfs = []
    for fileName in fileList:
        df = pd.read_csv(pathToRunData + os.sep + fileName)
        dfs.append(df)
    # Get the mean:
    accum = 0
    for df in dfs:
        accum += df[plotVariable]
    damean = accum/len(dfs)
        
    #print(plotVariable)
    plt.figure()
    plt.title("Experiment number = %d, Variable = %s, replications = %d." % (experimentNumber,plotVariable,len(dfs)))
        
    for df in dfs:
        #print(df.head())
        #plt.figure()
        #plt.title("Experiment number = %d, Variable = %s, replications = %d." % (experimentNumber,plotVariable,len(dfs)))
        plt.plot(df[plotVariable],'k',color = '0.75')
         
    # Now do the mean:
    plt.plot(damean,'k')
    plt.xlabel("Tick value (sampled every 1,000).")
    plt.ylabel("Wasp count in caste.")
    toPrint = id0name+' = '+str(parameterValues[0])+'\n'
    toPrint += id1name+' = '+str(parameterValues[1])+'\n'
    toPrint += id2name+' = '+str(parameterValues[2])+'\n'
    toPrint += id3name+' = '+str(parameterValues[3])
    plt.text(400,700,toPrint)
    # plt.show() The kernel dies when this is in.
    plt.savefig(outputFolder + os.sep + 'ExNum_'+str(experimentNumber)+ '_' + plotVariable + '_' + str(len(dfs)) + '_replications.pdf')
    # sok added 2016-05-09 in order to avoid having open lots of figures:
    plt.close()

In [ ]:
def processFileListTest(fileList,plotVariable,parameterValues,experimentNumber):
    '''
    We extract the say 30 replication files from the repositoryin main() 
    below. Here we get the list and we make a DataFrame of each file.
    Then we process each DataFrame to get a graph.
    '''
    #print('Entering processFileListList')
    #print('fileList: ',fileList)
    #print('plotVariable: ',plotVariable)
    #print('parameterValues: ', parameterValues)
    #print('experimentNumber: ',experimentNumber)
    dfs = []
    for fileName in fileList:
        df = pd.read_csv(pathToRunData + os.sep + fileName)
        #print(daPath)
        #df = pd.read_csv(daPath)
        dfs.append(df)
    return dfs

In [ ]:
df = pd.read_csv(pathToRunData + os.sep +'runDataLong0-0.csv')
df.head()

In [ ]:
def main(parameterList,experimentNumberList):
    exdict = makeExperimentParameterDict()

    for exnum in range(len(experimentNumberList)):
        experimentNumber = experimentNumberList[exnum]
        parameterValues = exdict[experimentNumber]
        
        fileList = os.listdir(pathToRunData)
        pattern = 'runDataLong' + str(experimentNumber) + '-'
        ourFiles = [x for x in fileList if pattern in x]
        for plotVariable in parameterList:
            processFileList(ourFiles,plotVariable,parameterValues,experimentNumber)
    return parameterValues 

In [ ]:
def mainTest(parameterList,experimentNumberList):
    '''
    parameterList is the first argument passed to main[Test] It
    is a list of the quantities we want graphed for each
    experiment in experimentNumberList.
    '''
    exdict = makeExperimentParameterDict()
    
    for exnumIdx in range(len(experimentNumberList)):
        # experimentNumber is the ID of the experiment we are
        # interested in.
        experimentNumber = experimentNumberList[exnumIdx]
        # Here are the values from its parameter list from the dictionary.
        parameterValues = exdict[experimentNumber]
        # OK, (1.0, 1.0, 1.0, 0.95) etc.
        #print('parameterValues: ',parameterValues)
        fileList = os.listdir(pathToRunData)
        pattern = 'runDataLong' + str(experimentNumber) + '-'
        ourFiles = [x for x in fileList if pattern in x]
        # OK, works: 
        #print(ourFiles)
        # ourFiles is now a list of the n replicants for a given experiment, e.g.
        # ['runDataLong0-0.csv', 'runDataLong0-1.csv', 'runDataLong0-2.csv', 
        #  'runDataLong0-3.csv', 'runDataLong0-4.csv']
        dfsList = []
        for plotVariable in parameterList:
            # ourFiles: list of the n replicants for a given experiment
            # plotVariable: member of the list passed as the first argument to main[Test]
            # parameterValues: (1.0, 1.0, 1.0, 0.95) etc. for this experiment
            # experimentNumber: 0, 1, etc.
            dfs=processFileListTest(ourFiles,plotVariable,parameterValues,experimentNumber)
            dfsList.append(dfs)
    # dfsList now is a list of length len(parameterList) of DataFrames, one for each file
    # processed.
    return dfsList #parameterValues #ourFiles

In [ ]:
def doit(numberOfRuns=2):
    graphList = list(range(0,numberOfRuns)) 
    # graphList is a list of integers corresponding to the 
    # experiments we want graphed.
    #print(graphList)
    fullList = ['TotalLiars', 	'Caste_0_Liars',	
                'signal_0_0', 'signal_0_0_mean_res',
                'signal_0_1', 'signal_0_1_mean_res', 
                'signal_0_2', 'signal_0_2_mean_res',
                'signal_1_0', 'signal_1_0_mean_res',
                'signal_1_1', 'signal_1_1_mean_res',
                'signal_1_2', 'signal_1_2_mean_res',
                'signal_2_0', 'signal_2_0_mean_res',
                'signal_2_1', 'signal_2_1_mean_res',
                'signal_2_2', 'signal_2_2_mean_res'
                ]
    returned = main(['Caste_2_Liars','Caste_1_Liars','Caste_0_Liars'],graphList)
    # returned is a list of length of the first argument to main[Test], that is,
    # the quantities we want graphed.
    return returned


In [ ]:
if __name__ == '__main__':
    x = numberOfRuns
    returned = doit(numberOfRuns=x)